In [ ]:
import cv2
from cv2 import destroyAllWindows
import mediapipe as mp
import numpy as np
import time
import speech_recognition as sr
import pyttsx3 
engine = pyttsx3.init()

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [ ]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle

In [ ]:
print("which Exercises you want to do \n Squat \n Dumbbell \n Shoulder raises \n plank")

In [ ]:
r=sr.Recognizer()
with sr.Microphone() as source:
    r.adjust_for_ambient_noise(source)

    print("please say sometning.....")

    audio=r.listen(source)

    print("Recognizing Now ....")
    try:
        Exercises=r.recognize_google(audio)
        print(Exercises)
        print("audio Recorder Successfully \n")
    except Exception as e:
        print("Error :"+ str(e))

    with open("recordedaudio.wav","wb") as f:
        f.write(audio.get_wav_data())
        
while True:
    if (Exercises == 'Squat' or Exercises == 'squat'):
        cap = cv2.VideoCapture(0)
        WIDTH = 1200
        HEIGHT = 1200
        cap.set(cv2.CAP_PROP_FPS, 25)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, WIDTH)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, HEIGHT)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Curl Counter Variables
        counter = 0
        stage = None
        feedback=''
        ## Setup Mediapipe Instance
        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.6) as pose:
            while cap.isOpened():
                ret, frame = cap.read()

                # ReColor Image to RGB
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False
        
                # Make Detection
                results = pose.process(image)

                # Recoloring Back to BGR
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
                # Extract Landmarks
                try:
                    landmarks = results.pose_landmarks.landmark
            
                    # Get Coordinates (Left)
                    shoulder_left = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                    elbow_left = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                    wrist_left = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
                    # Get Coordinates (Right)
                    shoulder_right = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                    elbow_right = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                    wrist_right = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
                    # Calculate Angle
                    angle_left = calculate_angle(shoulder_left, elbow_left, wrist_left)
                    angle_right = calculate_angle(shoulder_right, elbow_right, wrist_right)
            
                    # Visualize Angle
                    cv2.putText(image,str(angle_left),tuple(np.multiply(elbow_left, [640, 480]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2,cv2.LINE_AA)
                    cv2.putText(image,str(angle_right),tuple(np.multiply(elbow_right, [640, 480]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2,cv2.LINE_AA)
            
                    # Counter Logic
                    if angle_left > 160 and angle_right > 160:
                        stage = 'down'
                    if angle_left > 160 and angle_right > 160:
                        feedback = 'Almost there...Good !'
                    if (angle_left < 35 or angle_right < 35) and stage == 'down':
                        stage = 'up'
                        counter += 1
                    if stage == "up":
                        feedback = 'Good'
               
            
                except:
                    pass
        
                cv2.rectangle(image, (0,0), (1000,80), (700,400,75), -1)
        
                # Rep data
                cv2.putText(image, 'REPS', (15,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, str(counter), 
                            (10,60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        
                # Stage data
                cv2.putText(image, 'STAGE', (65,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, stage, 
                            (60,60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1.5, (45,45,45), 2, cv2.LINE_AA)
                cv2.putText(image, "FEEDBACK:",(200,13), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, feedback, ((200,50)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (45,45,45), 2, cv2.LINE_AA)
                
                engine.say(feedback) 
                engine.runAndWait()
                
                #Exercises name
                cv2.putText(image, 'Squat', (700,50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,0), 2, cv2.LINE_AA)
        
                # Render Detections
                mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,mp_drawing.DrawingSpec(color=(245, 117, 66),thickness=4,circle_radius=4),mp_drawing.DrawingSpec(color=(245, 66, 230),thickness=4,circle_radius=4))

                cv2.imshow('PERSIONAL GYM ASSITANC', image)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            cap.release()
            cv2.destroyAllWindows()
            
    if (Exercises == 'Dumbbell' or Exercises == 'dumbbell'):
        cap = cv2.VideoCapture(0)
        WIDTH = 1200
        HEIGHT = 1200
        cap.set(cv2.CAP_PROP_FPS, 70)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, WIDTH)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, HEIGHT)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Curl Counter Variables
        counter = 0
        stage = None
        feedback=''
        ## Setup Mediapipe Instance
        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.6) as pose:
            while cap.isOpened():
                ret, frame = cap.read()
                
                # ReColor Image to RGB
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False

                # Make Detection
                results = pose.process(image)

                # Recoloring Back to BGR
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                # Extract Landmarks
                try:
                    landmarks = results.pose_landmarks.landmark

                    # Get Coordinates (Left)
                    shoulder_left = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                    elbow_left = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                    wrist_left = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

                    # Get Coordinates (Right)
                    shoulder_right = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                    elbow_right = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                    wrist_right = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

                    # Calculate Angle
                    angle_left = calculate_angle(shoulder_left, elbow_left, wrist_left)
                    angle_right = calculate_angle(shoulder_right, elbow_right, wrist_right)

                    # Visualize Angle
                    cv2.putText(image,str(angle_left),tuple(np.multiply(elbow_left, [640, 480]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2,cv2.LINE_AA)
                    cv2.putText(image,str(angle_right),tuple(np.multiply(elbow_right, [640, 480]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2,cv2.LINE_AA)

                    # Counter Logic
                    if angle_left > 160 and angle_right > 160:
                        stage = 'down'
                    if angle_left > 160 and angle_right > 160:
                        feedback = 'Almost there...Good !'
                    if (angle_left < 35 or angle_right < 35) and stage == 'down':
                        stage = 'up'
                        counter += 1
                    if stage == "up":
                        feedback = 'Good rep!'


                except:
                    pass

                cv2.rectangle(image, (0,0), (1000,80), (700,400,75), -1)

                # Rep data
                cv2.putText(image, 'REPS', (15,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, str(counter), 
                            (10,60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)

                # Stage data
                cv2.putText(image, 'STAGE', (65,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, stage, 
                            (60,60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1.5, (45,45,45), 2, cv2.LINE_AA)
                cv2.putText(image, "FEEDBACK:",(200,13), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, feedback, ((200,50)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (45,45,45), 2, cv2.LINE_AA)
                
                engine.say(feedback) 
                engine.runAndWait() 
                #Exercises name
                cv2.putText(image, 'Dumbbell', (650,50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,0), 2, cv2.LINE_AA)
               
                
                # Render Detections
                mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,mp_drawing.DrawingSpec(color=(245, 117, 66),thickness=4,circle_radius=4),mp_drawing.DrawingSpec(color=(245, 66, 230),thickness=4,circle_radius=4))

                cv2.imshow('PERSIONAL GYM ASSITANC', image)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            cap.release()
            cv2.destroyAllWindows()

    if (Exercises == 'Shoulder raises' or Exercises == 'shoulder raises'):
        cap = cv2.VideoCapture(0)
        WIDTH = 1200
        HEIGHT = 1200
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, WIDTH)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, HEIGHT)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Curl Counter Variables
        counter = 0
        stage = None
        feedback=''
        ## Setup Mediapipe Instance
        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.6) as pose:
            while cap.isOpened():
                ret, frame = cap.read()

                # ReColor Image to RGB
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False

                # Make Detection
                results = pose.process(image)

                # Recoloring Back to BGR
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                # Extract Landmarks
                try:
                    landmarks = results.pose_landmarks.landmark

                    # Get Coordinates (Left)
                    elbow_left = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                    shoulder_left = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                    hip_left = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]

                    # Get Coordinates (Right)
                    elbow_right = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                    shoulder_right = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                    hip_right = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

                    # Calculate Angle
                    angle_left = calculate_angle(elbow_left, shoulder_left, hip_left)
                    angle_right = calculate_angle(elbow_right, shoulder_right, hip_right)

                    # Visualize Angle
                    cv2.putText(image,str(angle_left),tuple(np.multiply(shoulder_left, [640, 480]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2,cv2.LINE_AA)
                    cv2.putText(image,str(angle_right),tuple(np.multiply(shoulder_right, [640, 480]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2,cv2.LINE_AA)

                    # Counter Logic
                    if angle_left >= 85 and angle_right >= 85:
                        stage = 'down'
                    if angle_left >= 120 or angle_right >= 120:
                        feedback = 'Hands too Rsises.. !'
                        stage = 'down'
                    if 70 >= angle_left <= 110 and 70>= angle_right <= 110:
                        feedback = 'Almost there...Good !'
                    if (angle_left < 25 or angle_right < 25) and stage == 'down':
                        stage = 'up'
                        counter += 1
                    if stage == "up":
                        feedback = 'Good rep!'


                except:
                    pass

                cv2.rectangle(image, (0,0), (1000,80), (700,400,75), -1)

                # Rep data
                cv2.putText(image, 'REPS', (15,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, str(counter), 
                            (10,60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)

                # Stage data
                cv2.putText(image, 'STAGE', (65,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, stage, 
                            (60,60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1.5, (45,45,45), 2, cv2.LINE_AA)
                cv2.putText(image, "FEEDBACK:",(200,13), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, feedback, ((200,50)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (45,45,45), 2, cv2.LINE_AA)
                
                engine.say(feedback) 
                engine.runAndWait()
                #Exercises name
                cv2.putText(image, 'Sholulder Raises', (650,50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1, cv2.LINE_AA)                

                # Render Detections
                mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,mp_drawing.DrawingSpec(color=(245, 117, 66),thickness=4,circle_radius=4),mp_drawing.DrawingSpec(color=(245, 66, 230),thickness=4,circle_radius=4))

                cv2.imshow('PERSIONAL GYM ASSITANC', image)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            cap.release()
            cv2.destroyAllWindows()


    if (Exercises == 'plank' or Exercises == 'plank'):
        exercise_duration=int(input("Enter exercise duration : "))
        cap = cv2.VideoCapture(0)
        WIDTH = 1200
        HEIGHT = 1200
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, WIDTH)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, HEIGHT)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Squat Counter Variables
        counter = 0
        stage = None
        feedback = ''

        # Start the timer
        start_time = time.time()

        # Setup Mediapipe Instance
        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.6) as pose:
            while cap.isOpened():
                ret, frame = cap.read()

                # ReColor Image to RGB
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False

                # Make Detection
                results = pose.process(image)

                # Recoloring Back to BGR
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                # Extract Landmarks
                try:
                    landmarks = results.pose_landmarks.landmark

                    # Get Coordinates (Left)
                    thumb_left = [landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value].x,
                                     landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value].y]
                    wrist_left=[landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                    elbow_left = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                                  landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                    shoulder_left = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                    hip_left = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                                     landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                    knee_left = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                    ankel_left = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                    foot_index_left = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,
                                     landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
                
                    # Get Coordinates (Right)
                    thumb_right = [landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value].x,
                                     landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value].y]
                    wrist_right=[landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                                     landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                    elbow_right = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                                  landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                    shoulder_right = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                     landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                    hip_right = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                    knee_right = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                                     landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                    ankel_right = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                                     landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                    foot_index_right = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,
                                     landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
                
                    # Calculate Angle
                    left_wrist_angel = calculate_angle(thumb_left, wrist_left, elbow_left)
                    left_elbow_angel= calculate_angle(wrist_left, elbow_left, shoulder_left)
                    left_shoulder_angel= calculate_angle(elbow_left, shoulder_left,hip_left)
                    left_hip_angel= calculate_angle(shoulder_left,hip_left,knee_left)
                    left_knee_angel= calculate_angle(hip_left,knee_left,ankel_left)
                    left_ancel_angel= calculate_angle(knee_left,ankel_left,foot_index_left)
                    
                    right_wrist_angel = calculate_angle(thumb_right, wrist_right, elbow_right)
                    right_elbow_angel= calculate_angle(wrist_right, elbow_right, shoulder_right)
                    right_shoulder_angel= calculate_angle(elbow_right, shoulder_right,hip_right)
                    right_hip_angel= calculate_angle(shoulder_right,hip_right,knee_right)
                    right_knee_angel= calculate_angle(hip_right,knee_right,ankel_right)
                    right_ancel_angel= calculate_angle(knee_right,ankel_right,foot_index_right)

                    # Visualize Angle
                    """cv2.putText(image, str(angle_left),
                                tuple(np.multiply(elbow_left, [640, 480]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                                (255, 255, 255), 2, cv2.LINE_AA)
                    cv2.putText(image, str(angle_right),
                                tuple(np.multiply(elbow_right, [640, 480]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                                (255, 255, 255), 2, cv2.LINE_AA)"""

                    # Counter Logic
                    if left_elbow_angel < 90  and right_elbow_angel < 90:
                        feedback = 'elbow is not proper'
                    if left_elbow_angel > 100  and right_elbow_angel > 100:
                        feedback = 'elbow is greater'
                        
                    if left_shoulder_angel < 90  and right_shoulder_angel < 90:
                        feedback = 'shoulder is not proper'
                    if left_shoulder_angel > 100  and right_shoulder_angel > 100:
                        feedback = 'shoulder is not proper greater'
                    
                    if left_hip_angel > 185 and right_hip_angel > 185:
                        feedback = 'hip is not proper '
                        
                    if right_knee_angel < 170 and left_knee_angel < 170:
                        feedback = 'knee is not proper ' 
                    
                except:
                    pass

                cv2.rectangle(image, (0, 0), (1000, 80), (700, 400, 75), -1)
                
                cv2.putText(image, "FEEDBACK:",(200,13), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, feedback, ((200,50)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (45,45,45), 2, cv2.LINE_AA)
                engine.say(feedback) 
                engine.runAndWait()
                # Exercises name
                cv2.putText(image, 'plank', (700, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 2, cv2.LINE_AA)

                # Render Detections
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                          mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=4, circle_radius=4),
                                          mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=4, circle_radius=4))

                # Calculate exercise duration
                elapsed_time = time.time() - start_time
                remaining_time = exercise_duration - elapsed_time

                # Display the timer
                timer_text = f"{int(remaining_time)} seconds"
                cv2.putText(image, 'Remaining Time:', (20,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image,timer_text, 
                            (10,60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                cv2.imshow('PERSONAL GYM ASSISTANT', image)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            cap.release()
            cv2.destroyAllWindows()
        break